In [102]:
# pip install pyg-lib -f https://data.pyg.org/whl/nightly/torch-2.1.0+cu121.html

import torch
from torch_geometric.data import HeteroData
from torch_geometric import EdgeIndex
from torch_geometric.loader import LinkNeighborLoader, NeighborLoader
from torch_geometric.metrics import LinkPredMAP, LinkPredPrecision, LinkPredRecall

from tqdm import tqdm
# import os
# os.environ['PYDEVD_DISABLE_FILE_VALIDATION']='1'
# os.environ['KMP_DUPLICATE_LIB_OK']='True'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## HeteroData, train/test loaders

In [28]:
from utils.data_transformation import data_to_heterograph

data = data_to_heterograph('encoded_data.npz')
data

HeteroData(
  movie={ x=[11909, 200] },
  user={ x=[448798, 14] },
  (user, watched, movie)={
    edge_index=[2, 1288996],
    time=[1288996],
  },
  (movie, rev_watched, user)={
    edge_index=[2, 1288996],
    time=[1288996],
  }
)

In [30]:
# vary sparse interaction matrix
1288996 / (11909 * 448798)

0.0002411711540184161

## MetaPath2Vec

In [35]:
from utils.node_representation import Metapath2Vec

mp2v = Metapath2Vec(train_data.edge_index_dict, train_data.num_nodes_dict, device=device)
mp2v.train()

users_emb = mp2v.get_embeddings('user').numpy()
movies_emb = mp2v.get_embeddings('movie').numpy()

## GNN

In [ ]:
data['user'].x = torch.cat([data['user'].x, users_emb], dim=1)
data['movie'].x = torch.cat([data['movie'].x, movies_emb], dim=1)

In [ ]:
from utils.model import GNN

gnn_model = GNN(data.metadata(), hidden_channels=64, decoder='IP',
                dropout_encoder_p=0.2, dropout_decoder_p=0.4)

optimizer = torch.optim.Adam(gnn_model.parameters(), lr=0.01)

# train()

# test()